In [ ]:
######################################################################################################
# Molina F., Chassin J., Camus H. (2024) Mistral AI for Developers Build Innovative Apps with Gen AI #
#                                                                                                    #
# This Notebook is a companion to the book. It provides the code examples and instructions needed to #
# replicate the concepts and techniques discussed in the book for integrating Mistral LLM into your  #
# projects and Google Cloud. Code for educational purpose.                                           #
######################################################################################################

In [ ]:
# install the required librairies
!pip -q install transformers -U > /dev/null
!pip -q install bitsandbytes -U > /dev/null
!pip -q install accelerate -U > /dev/null

In [ ]:
#reboot Colab
import os
os.kill(os.getpid(), 9)

In [ ]:
#necessary packages
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch
import accelerate

In [ ]:
# Load the model Quantized, this requires a GPU T4 minimum (click on the top right corner, change runtime and select T4 GPU)
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"
config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.max_position_embeddings = 8096
quantization_config = BitsAndBytesConfig(
bnb_4bit_compute_dtype=torch.bfloat16,
load_in_4bit=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_use_double_quant=True,
llm_int8_enable_fp32_cpu_offload=True
)

In [ ]:
# Load the model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
model_name_or_path,
config=config,
trust_remote_code=True,
quantization_config=quantization_config,
device_map="auto",
offload_folder="./offload"
)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Let's prompt the model
prompt = "[INST] What are the 3 main groups that are representative of the french touch electro music scene? [/INST] "
print(f"""###prompt:\n{prompt}\n### Mistral 7B Instruct:""")
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt= True)
output = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=256,
    temperature=0.8,
    do_sample=True,
    top_k=20,
    top_p=0.4,
    repetition_penalty=1.2
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###prompt:
[INST] What are the 3 main groups that are representative of the french touch electro music scene? [/INST] 
### Mistral 7B Instruct:
1. Deep House: This subgenre is characterized by its deep, rolling basslines and soulful vocals. It often features elements of disco, funk, and jazz fusion.
2. Techno: French Touch techno tends to be more melodic and atmospheric than traditional techno, with a focus on synth-based sounds and intricate production techniques.
3. Electrohouse: This subgenre combines elements of house and electronic dance music (EDM) to create high-energy tracks with strong beats and catchy hooks.</s>


In [ ]:
text = "<s> [INST]  What are the 3 main groups that are representative of the french touch electro music scene? [/INST]"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=256)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  What are the 3 main groups that are representative of the french touch electro music scene? [/INST] The French touch electro music scene is known for its unique sound and style, which combines elements of electronic dance music (EDM), house, and disco music. There are several sub-genres and styles within the French touch scene, but three main groups that are representative of this scene are:

1. Deep House: Deep house is a sub-genre of house music that is characterized by its use of deep, rolling basslines, soulful vocals, and atmospheric pads. It often features a more laid-back, groove-oriented sound compared to other sub-genres of French touch.
2. Tech House: Tech house is a sub-genre of house music that combines elements of techno and house music. It is characterized by its use of complex, intricate rhythms, deep basslines, and atmospheric pads. Tech house often features a more upbeat, danceable sound compared to deep house.
3. Disco House: Disco house is a sub-genre of hous